## Damares Source to Staging [Orders]
Created By : Rishad Harisdias Bustomi

Version : 1.0

Last Modified : 2023-03-29

This purpose of this etl job is to fetch excel data from FTP and then ingest it to Big Query staging database

In [25]:
# Import Needed Library

from google.cloud import bigquery
import pandas as pd
import ftplib
import os

In [ ]:
# Not Runnable
server = ftplib.FTP()
server.connect('10.10.2.123', 5556)
server.login('user','123456')

filenames = server.nlst()

for filename in filenames:
    local_filename = os.path.join('C:\\Users\\20220637\\Documents\\GitHub\\bie-bootcamp\\dataset\\', filename)
    file = open(local_filename, 'wb')
    ftp.retrbinary('RETR '+ filename, file.write)

    file.close()

ftp.quit()

In [2]:
# Connect to Big Query Client
key_path = r'C:\Users\20220637\Documents\Me\Binar\BigQuery\Credential\bie-testing-22f0e30d8f95.json'
client = bigquery.Client.from_service_account_json(key_path)

# Specify table
table_id = 'bie-testing.superstore_stg.orders'

# List source file
source_folder = r"C:\Users\20220637\Documents\GitHub\bie-bootcamp\dataset\orders"
dir_list = os.listdir(source_folder)

In [31]:
# Ingest To Bigquery
df_total = pd.DataFrame()
for file in dir_list:
    source = os.path.join(source_folder,file)
    df = pd.read_excel(source)
    columns = ['row_id','order_id','order_date','ship_date','ship_mode','customer_id','country','city','state','postal_code','region','product_id','sales','quantity','discount','profit']
    df.columns = columns
    df_total = concat([df_total,df], axis=0)

result = insert_to_bigquery(df_total, table_id)
print(result)

NameError: name 'concat' is not defined

### Function

In [28]:
def insert_to_bigquery(df, table_id):
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE",
    )

    job = client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )
    job.result()

    table = client.get_table(table_id)  # Make an API request.
    result = f"Loaded {table.num_rows} rows and {len(table.schema)} columns to {table_id}"
    return result